# DSCI 575 - Advanced Machine Learning

# Lab 1: Text generation using Markov Models

## Table of contents
- [Submission guidelines](#sg) (4%)
- [Exercise 1: Warm-up](#1) (20%)
- [Exercise 2: Character-based Markov model of language](#2) (20%)
- [Exercise 3: Stationary distribution and other fun stuff](#3) (30%)
- [Exercise 4: Word-based Markov model of language](#4) (26%)
- [Submission to Canvas](#sc)

## Submission instructions <a name="si"></a>
<hr>
rubric={mechanics:4}

You will receive marks for correctly submitting this assignment. 

To correctly submit this assignment follow the instructions below:

- Push your assignment to your GitHub repository. 
- Add a link to your GitHub repository here: https://github.ubc.ca/MDS-2020-21/DSCI_575_lab1_lhabashy 
- Upload an HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).

[Here](https://github.com/UBC-MDS/public/tree/master/rubric) you will find the description of each rubric used in MDS.

**NOTE: The data you download for use in this lab SHOULD NOT BE PUSHED TO YOUR REPOSITORY. You might be penalised for pushing datasets to your repository. I have seeded the repository with `.gitignore` and hoping that it won't let you push CSVs.**

<br><br><br><br>

## Imports <a name="im"></a>

In [1]:
import os
import re
import sys
from collections import Counter, defaultdict
from urllib.request import urlopen

import numpy as np
import numpy.random as npr
import pandas as pd

<br><br><br><br>

## Exercise 1: Warm-up <a name="1"></a>

This exercise will get you thinking about how to generate text using Markov models. In the lecture, we built a Markov model of language on a toy corpus using words. In this exercise, you will build a letter or character-based Markov model of language with a tiny corpus. The goal of such a model is to predict next character given a sequence of characters. 

In NLP, a Markov model of language is also referred to as **an n-gram language model**. In this exercise we'll focus on a character-based bigram (2-gram) language model. We will use the variable `n=1` to denote that we are only considering the current character in the sequence to predict the next character. In the next exercise, you'll explore different values for `n` in your n-gram language model. 
 
As we saw in class, to build a bigram model, we need bigram frequency counts, i.e., counts of all 2-letter sequences in your corpus. Below I am providing you some starter code to create bigram frequency counts of letters in a tiny corpus with 6 words. Our goal is to build a character-based bigram language model with this corpus, where the set of states is the unique letters in the corpus.  

> When we say n-gram we're now referring to the last `n` characters, not the last `n` words. In general, the term n-gram can refer to characters or word; see the [Wikipedia article](https://en.wikipedia.org/wiki/N-gram).


> Recall that in [DSCI 512 lab1](https://github.ubc.ca/MDS-2020-21/DSCI_512_alg-data-struct_students/blob/master/solutions/lab1/lab1.ipynb), you implemented a Markov model of language and generated text (without actually knowing the details about the model). You pre-computed the frequencies for every possible `n` gram as follows.  

In [2]:
corpus = "to be or not to be"  # our tiny corpus
n = 1  # for bigrams
circ_corpus = corpus + corpus[:n]
frequencies = defaultdict(Counter)
for i in range(len(circ_corpus) - n):
    frequencies[circ_corpus[i : i + n]][circ_corpus[i + n]] += 1
frequencies

defaultdict(collections.Counter,
            {'t': Counter({'o': 2, ' ': 1}),
             'o': Counter({' ': 2, 'r': 1, 't': 1}),
             ' ': Counter({'b': 2, 'o': 1, 'n': 1, 't': 1}),
             'b': Counter({'e': 2}),
             'e': Counter({' ': 1, 't': 1}),
             'r': Counter({' ': 1}),
             'n': Counter({'o': 1})})

<br><br>

### 1.1 Visualizing character bigram counts as a co-occurrence matrix
rubric={accuracy:4} 

**Your tasks:**

1. Show the bigram frequencies in `frequencies` variable above as a pandas DataFrame, sorting the column and row labels alphabetically, where
    * column labels and row indices are unique characters in the corpus, and 
    * the value in each cell $a_{ij}$ represents how often the character $i$ precedes character $j$ in our corpus.
    
> Note: Fill in the NaN values with zeros. 

In [4]:
### Solution_1_1_1

### YOUR ANSWER HERE
freq_df = pd.DataFrame(frequencies).transpose()
freq_df = freq_df.fillna(0)
freq_df

,o,,r,t,b,n,e
t,2.0,1.0,0.0,0.0,0.0,0.0,0.0
o,0.0,2.0,1.0,1.0,0.0,0.0,0.0
,1.0,0.0,0.0,1.0,2.0,1.0,0.0
b,0.0,0.0,0.0,0.0,0.0,0.0,2.0
e,0.0,1.0,0.0,1.0,0.0,0.0,0.0
r,0.0,1.0,0.0,0.0,0.0,0.0,0.0
n,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<br><br>

### 1.2 Transition matrix
rubric={accuracy:6}


**Your tasks:**

1. Given the frequencies in 1.1, compute the transition matrix (i.e., the conditional probability distribution for every possible bigram). Visualize the transition matrix as a pandas DataFrame.  
2. How many unique states are there in your Markov model? 

> Recall that the transition matrix $T$ is a square matrix and the number of rows/columns is equal to the number of states. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

In [5]:
### Solution_1_2_1

### YOUR ANSWER HERE
trans_df = freq_df.div(freq_df.sum(axis=1), axis=0)
trans_df

,o,,r,t,b,n,e
t,0.666667,0.333333,0.00,0.00,0.0,0.0,0.0
o,0.000000,0.500000,0.25,0.25,0.0,0.0,0.0
,0.200000,0.000000,0.00,0.20,0.4,0.2,0.0
b,0.000000,0.000000,0.00,0.00,0.0,0.0,1.0
e,0.000000,0.500000,0.00,0.50,0.0,0.0,0.0
r,0.000000,1.000000,0.00,0.00,0.0,0.0,0.0
n,1.000000,0.000000,0.00,0.00,0.0,0.0,0.0


In [ ]:
### YOUR ANSWER HERE

In [8]:
### Solution_1_2_2

### YOUR ANSWER HERE
states = np.unique(list(frequencies.keys()))
print("States:", states)
S = len(states)
print("Number of states:", S)

States: [' ' 'b' 'e' 'n' 'o' 'r' 't']
Number of states: 7


<br><br>

### 1.3 Probability of sequences
rubric={reasoning:4}

**Your tasks:**

Suppose the probability of starting a sequence with letter $b$ is $0.4$ (i.e., $\pi_0$ of state $b$ = 0.4). Given the transition matrix in 1.2, calculate the probabilities for the following sequences. 

1. "be or"
2. "beet"

**Solution_1_3**
### YOUR ANSWER HERE

$P($"be or"$) = P(b|b=0.4) \times P(e|b) \times P($ $|e) \times P(o|$ $) \times P(r|o)$
<br> $\implies 0.4 \times 1 \times 0.5 \times 0.2 \times 0.25 = 0.01$

$P($"beet"$) = P(b|b=0.4) \times P(e|b) \times P(e|e) \times P(t|e)$
<br> $\implies 0.4 \times 1 \times 0 \times 0.5 = 0$

<br><br>

### 1.4 Generate sequences
rubric={reasoning:6}

**Your tasks:**

1. Given the transition matrix above, explain in words how would you generate a sequence of length `seq_len` given the seed letter `t`. You don't have to, but you are welcome to write code for this. 
2. Give a few example applications for word- or character-level text generation. 

**Solution_1_4_1**

### YOUR ANSWER HERE

> Starting at state `t`, we would sample the next letter from the letter's probability distribution, computed in the transition matrix. Once the next state in sampled, we can transition into it. Once we're in the new state -- suppose it's the letter "e" since it has the highest probability of getting sampled -- we can use the Markov assumption and forget the past states. That is, we now need to sample from the probability distribution of state `e`, a sampling methof independent of our previous states. I would repeat this process until the generated sequence of text reaches `seq_len`.

In [ ]:
### YOUR ANSWER HERE

**Solution_1_4_1**

### YOUR ANSWER HERE

> A few applications for word- or character-level text generation may include:
<br> - iMessages autocomplete text suggester 
<br> - Google home 
<br> - Autofill search bars
<br> - Automatic replies in Emails
<br> - Discord chatbots

<br><br><br><br>

## Exercise 2: Character-based Markov model of language <a name="2"></a>

Recall Markov Model of language from [DSCI 512 lab1](https://github.ubc.ca/MDS-2020-21/DSCI_512_alg-data-struct_students/blob/master/solutions/lab1/lab1.ipynb). In that lab, you wrote a class `MarkovModel` to `fit` an n-gram Markov model and generated text. Now that you know what Markov models are, we will start with what you did in DSCI 512 lab1 and build on it. 

The starter code below uses the hyperparameter `n`, where our "state" of the Markov chain is the last `n` characters of a give string. In Exercise 1, we worked with `n=1` (bigram model) and our "state" of the Markov chain was a single character and each character was dependent on the last one character. When `n=3`, it means that the probability distribution over the _next character_ only depends on the _preceding 3 characters_. 

> Note that `n` in n-gram doesn't exactly correspond to the variable `n` we are using in the implementation below. For 2-gram (bigram) the value of the variable `n` is 1, and for 4-gram the value of `n=3`. 

We train our model from data by recording every occurrence of every n-gram and, in each case, recording what the next letter is. Then, for each n-gram, we normalize these counts into probabilities just like we did with naive Bayes. The `fit` function below implements these steps.

To generate a new sequence, we start with some initial seed at least of length `n` (here, we will just use the first `n` characters in the training text, which are saved at the end of the `fit` function). Then, for the current n-gram we will look up the probability distribution over next characters and sample a random character according to this distribution.

Attribution: assignment adapted with permission from Princeton COS 126, [_Markov Model of Natural Language_]( http://www.cs.princeton.edu/courses/archive/fall15/cos126/assignments/markov.html). Original assignment was developed by Bob Sedgewick and Kevin Wayne. If you are interested in more background info, you can take a look at the original version. The original paper by Shannon, [A Mathematical Theory of Communication](http://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf), essentially created the field of information theory and is one of the best scientific papers ever written (in terms of both impact and readability).  

In [11]:
class MarkovModel:
    def __init__(self, n):
        """
        Initialize the Markov model object.

        Parameters:
        ----------
        n : int
            the size of the ngram
        """
        self.n = n

    def fit(self, text):
        """
        Fit a Markov chain and create a transition matrix.

        Parameters
        ----------
        text : str
            text to fit the Markov chain
        """

        # make text circular so markov chain doesn't get stuck
        circ_text = text + text[: self.n]

        # count the number of occurrences of each letter following a given n-gram
        frequencies = defaultdict(Counter)
        for i in range(len(text)):
            frequencies[circ_text[i : i + self.n]][circ_text[i + self.n]] += 1.0

        # normalize the frequencies into probabilities (separately for each n-gram)
        self.probabilities_ = defaultdict(dict)
        for ngram, counts in frequencies.items():
            self.probabilities_[ngram]["symbols"] = list(counts.keys())
            probs = np.array(list(counts.values()))
            probs /= np.sum(probs)
            self.probabilities_[ngram]["probs"] = probs

        # store the first n characters of the training text, as we will use these
        # to seed our `generate` function
        self.starting_chars = text[: self.n]
        self.frequencies_ = frequencies  # you never know when this might come in handy

    def generate(self, seq_len):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
        s = self.starting_chars
        while len(s) < seq_len:
            probs = self.probabilities_[s[-self.n :]]
            s += npr.choice(probs["symbols"], p=probs["probs"])
        return s

<br><br>

Let's test our code. 

In [13]:
# Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm
data_url = "http://www.gutenberg.org/files/2591/2591-0.txt"

corpus = urlopen(data_url).read().decode("utf-8")
corpus = corpus[2000:]
model = MarkovModel(n=3)
model.fit(corpus)
print(model.generate(500))

E WOLF AND THAT FOX

Sect Gutenber; anot for live younday,
days could gave barge to away.’ But, had that the down shallent began escape and or jour he woman, replie wake in his fore and hers lod rest!’

‘What has ful this found the way cannot
find she read.’ She lanket
and shorse use
measten snes; I mutter mothe
dantant’;
and was del servanted.’
‘But
fell, no he homess, and with of cat, and which give cripe a long ther
had
king is very sittle daught
of not skillowever, ‘and them, 


<br><br>

### 2.1 "Circular" version of the text
rubric={reasoning:5}

**Your taks:**

Why do we need to use a "circular" version of the text in the `fit` function? What could go wrong if we didn't do that, and instead used the original `text` (instead of `circ_text`) and made the loop:  
```
for i in range(len(text)-self.n):
```
which allow `fit` to run without crashing?

> Hint: the problem would arise during `generate`, not during `fit`.



**Solution_2.1**

### YOUR ANSWER HERE

> The main reason we use a "circular" version `circ_text` is that the Markov chain does not get stuck in any state with no characters. That is, we ensure the ability to transition with having non-zero transition probabilities. This also imposes the irreducibity assumption which assumes the graoh does not get stuck at any state, and possibly the assumption of aperiodicity, which suggests no repeation of the same sequences occur. Omitting the use of this text version, we are exposing the chain in getting stuck in a loop when generating the text since the length of the generated text would be smaller than the length of the desired length of the sequence.  

### (Optional) 2.2 Smoothing
rubric={reasoning:1}

The description above mentioned a connection to naive Bayes. When discussing naive Bayes we briefly mentioned [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing). Is there something analogous to Laplace smoothing that we could do here? If so, describe what it would entail and what effect it would have. (You don't have to implement it.)

**Solution_2.2**
### YOUR ANSWER HERE

> Analogous to Laplace smoothing, we could impose some filters and additive smoothing into the Markov model using interpolation or absolute discounting. The effect could be observed in the smoothness when testing various window sizes and n-grams.

<br><br>

### 2.3 States, state space, and transition matrix
rubric={reasoning:15}

Let's consider the Markov chain interpretation of what we've just done. Let's define a state as the previous $n$ characters "emitted" by our chain. Let's assume our vocabulary size (the number of possible characters) is $V$; for example, $V=26$ if we were only using lowercase letters. We can compute this in our case:

In [ ]:
print("Vocabulary size = %d" % len(np.unique(list(corpus))))

But let's just stick with a general $V$ for now. Call the number of possible states $S$. Let's consider the _transition matrix_ $T$ of this Markov chain. The transition matrix is a square matrix and the number of rows/columns is equal to the number of states, so in our case it is $S \times S$. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

**Your tasks:**

Answer the following questions:

1. What is the number of possible states, $S$, in terms of $V$ and $n$? (You might not encounter all these states in practice, if not all $n$-grams appear in the training data, but let's ignore that for now.)
2. Give two examples states in the state space for $n=3$.
3. In our application above when $n>1$ _not all transitions are possible_. This means $T$ will contain a bunch of zeros. Why?
4. Again for $n>1$ what is the maximum number of nonzero elements that $T$ could have? Answer in terms of $V$ and $n$. 
5. Is the state space discrete or continuous?

### YOUR ANSWER HERE

> 1. $V^n$
2. {aaa, abc}
3. When $n>1$ not all transitions are possible since we cannot transition some states to others. Suppose we wanted to switch from state $abc$ to state $aaa$. This would not be possible since we can only transition $bc\alpha$ for some $\alpha$ from all permutations of $bc\alpha$ in the corpus.
4. The maximum number of nonzero elements that $T$ could have is $V^{n+1}$. The number of nonzero elements in every row is all the possible states to transtition into, which is equivalent to the number of states. That is, $S=V^n$, the number of columns. Therefore, nonzero elements will be at maximum $S\times V = V^n \times V = V^{n+1}$. This makes up a very small proportion of the total number of elements $V^{2n}$.
5. Discrete

<br><br><br><br>

### Exercise 3: Stationary distribution and other fun stuff <a name="3"></a>

The code below computes the transition matrix for you for the Shakespeare data with `n=1`. Consider this transition matrix for the next few questions.

In [9]:
# The Project Gutenberg eBook of The Complete Works of William Shakespeare, by William Shakespeare
data_url = "http://www.gutenberg.org/files/100/100-0.txt"
shakespeare_text = urlopen(data_url).read().decode("utf-8")
shakespeare_text = shakespeare_text[4000:]

In [12]:
states = np.unique(list(shakespeare_text))
print("States:", states)
S = len(np.unique(list(shakespeare_text)))
print("Number of states:", S)

model = MarkovModel(n=1)
model.fit(shakespeare_text)

# implementation note: since len(model.probabilities_[state]["probs"]) might not equal S
# for all letters, we need to be careful and do a reverse-lookup for the actual letters
# the rest of the transition probabilities are just zero (if they don't appear)
lookup = dict(zip(states, list(np.arange(S, dtype=int))))

T = np.zeros((S, S))  # transition matrix
for i, state in enumerate(states):
    for j, letter in enumerate(model.probabilities_[state]["symbols"]):
        T[i, lookup[letter]] = model.probabilities_[state]["probs"][j]

print("Number of nonzero transition probabilities: %d/%d" % (np.sum(T > 0), T.size))

States: ['\t' '\n' '\r' ' ' '!' '"' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.' '/'
 '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '?' '@' 'A' 'B' 'C' 'D'
 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V'
 'W' 'X' 'Y' 'Z' '[' '\\' ']' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '|'
 '}' 'Æ' 'É' 'à' 'â' 'æ' 'ç' 'è' 'é' 'ê' 'ë' 'î' 'œ' '—' '‘' '’' '“' '”']
Number of states: 107
Number of nonzero transition probabilities: 2647/11449


<br><br>

### 3.1 Stationary distribution conditions 
rubric={reasoning:5}

Under mild assumptions, a Markov chain has a _stationary distribution_ which is the probability of finding yourself in the same state after the chain is run for a long time. These assumptions are:

- "Irreducibility" (doesn’t get stuck in part of the graph)
- "Aperiodicity" (doesn’t keep repeating same sequence).

Give a short explanation for why our Markov chain might satisfy these assumptions.

### YOUR ANSWER HERE

> I would suspect that Markoc chain is irreducible and aperiodic since I'm assuming the text is written in proper English. As such, it's probably unlikely the chain gets stuck at a given state. Also, the states probability distributions seem to have a lot of variation. Looking at the transition matrix $T$, it seems we're able to transition freely between all 107 states, which would satisfy the irreduciblity assumption. However, the third state \r is always followed by \n (with probability 1). Inspecting the shakespearen text, we see that \r is indeed only ever followed by \n. Though seeing the following transition is a little unfeasible, we assume an exit is possible, to a space character perhaps. Thus based on the transition matrix, once if we're in that state (state 2) and cannot leave, this markov chain would be reducible. Seeing how this may be an anomoly, we can make the assumption that the Markov chain is indeed irreducible. We can also check the diagonals of the transition matrix are 1 but that aline may not enough to prove irreduciblity.

In [33]:
### YOUR ANSWER HERE
#pd.DataFrame(T)

In [32]:
#shakespeare_text

<br><br>

### 3.2 Stationary distribution for the `shakespeare_text`
rubric={accuracy:10}

It's not true in general but in this particular case we actually already know the stationary distribution -- it is just the relative frequency that these n-grams occur in the training data (you are welcome to think about this deeply, but you don't have to). (Recall that we are assuming n=1 here.)

**Your tasks:** 

1. Calculate the stationary distribution for the `shakespeare_text` by calculating the relative frequencies for each state in the corpus (unique letter in this case, as we are assuming `n=1`).
2. Show empirically that this is a stationary distribution. 

In [76]:
text_len = len(shakespeare_text)
freq_df = np.array([shakespeare_text.count(i) / text_len for i in states])
pd.DataFrame(freq_df, columns=['Freq'])

,Freq
0,5.251253e-07
1,2.983482e-02
2,2.983482e-02
3,1.845093e-01
4,1.490131e-03
...,...
102,2.888189e-04
103,6.336512e-05
104,2.995840e-03
105,8.191955e-05


In [77]:
# Code adapted from lecture 1 notes

def stationary_dist(pi_0, T, time_step=20):
    print('pi_0 =', pi_0)
    pi_time_step = pi_0@np.linalg.matrix_power(T,time_step)
    print('pi_%d = %s'%(time_step,pi_time_step))
    if not np.allclose(pi_time_step@T, pi_time_step): 
        print('Not steady state yet: pi_%d@T != pi_%d'
              %(time_step,time_step))
    else:     
        print('Steady state: pi_%d@T == pi_%d'%(time_step,time_step))
               
stationary_dist(freq_df, T, time_step = 1)

pi_0 = [5.25125321e-07 2.98348201e-02 2.98348201e-02 1.84509258e-01
 1.49013062e-03 1.40033419e-05 3.50083547e-07 1.75041774e-07
 1.15527571e-05 2.49189469e-03 5.74137018e-05 5.72386600e-05
 4.55108612e-06 1.62132443e-02 1.01104129e-03 1.48939544e-02
 2.10050128e-06 2.87068509e-05 6.16147043e-05 3.97344826e-05
 2.87068509e-05 3.37830623e-05 2.03048458e-05 2.71314749e-05
 1.54036761e-05 2.59061825e-05 1.61038432e-05 8.24796838e-04
 2.99093879e-03 1.94313873e-03 1.75041774e-07 7.78095693e-03
 2.32087888e-03 3.28413376e-03 2.30232445e-03 6.31375678e-03
 1.98112279e-03 1.82165974e-03 2.94070180e-03 9.06016221e-03
 3.25052574e-04 1.00929087e-03 3.81328504e-03 2.53985614e-03
 4.35959042e-03 4.84515630e-03 1.84424013e-03 2.06199209e-04
 4.22288279e-03 5.34472552e-03 6.74610996e-03 2.24858663e-03
 5.58033175e-04 2.91129478e-03 6.42403310e-05 1.27045319e-03
 9.87235604e-05 6.09670498e-04 1.75041774e-07 6.09845540e-04
 9.38048865e-04 1.75041774e-07 4.64706152e-02 8.88109447e-03
 1.28183091e-02 2

<br><br>

### (optional) 3.3 Stationary distribution using eigenvalue decomposition
rubric={reasoning:1}

You can compute the stationary distribution in another ways. 

**Your tasks**

1. Calculate stationary distribution using eigenvalue decomposition of the transition matrix. Compare your result from part 3.2 with the result below. Do they agree with each other?

> We haven't explicitly talked about this in class but at this point in the program, I believe that you are independent enough to do it on your own. 

In [ ]:
### Solution_3_3_1
### YOUR ANSWER HERE

<br><br>

### 3.4 Finding probability of occurrences of patterns 
rubric={reasoning:15}

Let's consider the conditional probability that a lower case vowel comes 3 characters later given that the current letter is "a". In other words, we're searching for the pattern `axxv` where `v` is a lower case vowel (defined as a,e,i,o,u,y) and `x` is any character and `a` is literally "a". 

Let's use `n=1`.

**Your tasks:**
1. It turns out we can estimate this probability directly from the transition matrix. While $T$ gives us the probabilities one step ahead, $T\times T$ gives us the probabilities 2 steps ahead, etc. (If you want to think about this, you should be able to convince yourself it's true by contemplating what matrix multiplication really means, but this is optional.) So, taking $T^3$ gives us the transition probabilities for 3 steps later. Compute $T^3$ and find the estimated probability that we're looking for. 
2. We could also estimate this probability directly from the data, by just seeing how frequently this pattern occurs. Do this. How well does it match your above results? What does this tell us about how reasonable our  Markov assumption is?
3. What if we increased `n` and repeated part (1), would you expect to get closer to the answer from part (2)? You are welcome to try this but you don't have to - the goal is just to think about it.

> Note for 3.4.1: you should NOT use `T**3`, as that is element-wise exponentiation rather than a matrix power. You can get $T^3$ using `numpy.linalg.matrix_power(T,3)` or just `T@T@T`.

> This question might be a bit tricky. But hopefully it will help you understand Markov models better. 

In [43]:
### Solution_3_4_1

### YOUR ANSWER HERE
vowels = ['a', 'e', 'i', 'o', 'u','y']
unique_states = list(states)
T_df = pd.DataFrame(T, columns=unique_states, index=unique_states)
T_df_3 = T_df @ T_df @ T_df
T_df_3.head()

,\t,\n,\r,,!,"""",$,%,&,',...,é,ê,ë,î,œ,—,‘,’,“,”
\t,1.382155e-07,0.000963,0.016311,0.144041,0.001256,0.000011,4.901805e-07,0.000000e+00,0.000008,0.002744,...,0.000006,5.325955e-07,1.810708e-07,1.108755e-07,3.487511e-06,0.000143,0.000029,0.003378,0.000037,0.000039
\n,1.523956e-07,0.002177,0.055919,0.171032,0.000947,0.000008,2.654925e-07,4.506554e-07,0.000006,0.002131,...,0.000005,3.974693e-07,1.507668e-07,6.466956e-08,1.403737e-06,0.000205,0.000136,0.002883,0.000216,0.000061
\r,2.977522e-07,0.172250,0.002177,0.087209,0.000134,0.000023,2.533663e-06,0.000000e+00,0.000019,0.001342,...,0.000004,1.194201e-06,1.078453e-08,7.291188e-09,4.261705e-08,0.000023,0.000073,0.001600,0.000104,0.000008
,5.009547e-07,0.006664,0.025838,0.178863,0.001576,0.000012,2.520116e-07,3.491315e-07,0.000010,0.002564,...,0.000007,7.597677e-07,2.190216e-07,1.151768e-07,1.449376e-06,0.000294,0.000037,0.003121,0.000041,0.000073
!,8.592789e-08,0.010453,0.094844,0.240122,0.000706,0.000006,3.844704e-07,9.672549e-09,0.000008,0.001662,...,0.000003,3.255726e-07,1.088770e-07,1.045138e-07,4.821439e-07,0.000093,0.000368,0.002761,0.000601,0.000011


In [44]:
### YOUR ANSWER HERE
# search
T_df_3.loc['a', vowels].sum().round(4)

0.2551

In [47]:
### Solution_3_4_2

### YOUR ANSWER HERE

# using 4-gram method

tot = 0
letter_a = 0
for i in range(len(shakespeare_text)-3):
    if shakespeare_text[i] == 'a':
        letter_a +=1
        if shakespeare_text[i+3] in vowels:
            tot +=1
round(tot/letter_a, 4)

0.2548

> The result matches the above results almost perfectly. This indicates the Markov chain assumptions were pretty reasonable as the model is able to predict the future, regardless of the past, at high accuracy.

**Solution_3_4_3**

### YOUR ANSWER HERE

> Yes, if we increased `n`, I would expect to get closer to the answer from part (2) because that would mean incorporating more context and information into the Markov chain computations. We could get better predictions as the model would be closer to memorizing the data, though it would take longer.

<br><br><br><br>

## Exercise 4: Markov model of language with words <a name="4"></a>

In this exercise we'll continue with the Markov model of language, but we'll work with _words_ instead of _characters_. The `MarkovModel` code stays the same. Just remember that now we are dealing with words and not characters. 

When we say n-gram we're now referring to `n` words, not `n` characters.

One concern with words is that, in a sense, we have less examples to work with. For example, with character n-grams and `n=3` we have a lot of examples of seeing the character combination "per" and checking what character follows it, but with words even with `n=1` (the smallest nontrivial value of `n`), we might only have one example of the word "persuade" in our corpus. You can imagine that the number of "training examples" only gets smaller if `n`>1. This is something to keep in mind when deciding what values of `n` might be reasonable.

Another issue is that the number of states could explode when you are working with words. For example, with character bigram model our states were all unique characters in the text (107 in our example above). For word bigram model, it's going to be the number of unique words in the corpus, which would be a much bigger number. You can imagine that for a large corpus and bigger values of `n`, the number of states could explode.  

Finally, we need to preprocess the text (i.e., tokenization) before creating a word-based n-gram model. To accomplish this preprocessing, we will use the popular python package [NLTK](http://www.nltk.org/) (Natural Language ToolKit), which we have seen in class. If you are using the course conda environment, you should already have the package in your conda environment. 

You might need to install the NLTK data files with the following command in your conda environment in the terminal.

```
python -m nltk.downloader 'punkt'
```

### 4.1 Word-based n-gram language model
rubric={accuracy:8,reasoning:4}

The first step is to break apart the text string into words. NLTK's `word_tokenize` will turn our text into a lists of "tokens" which we will use as our language units. This tokenizing process removes whitespace. So, in `generate`, you should add a space character back in between tokens. This won't look quite right because you'll have spaces before punctuations, but it's good enough for now.

**Your tasks:**

1. Implement a word-based Markov model of language.
2. Train your word-based Markov model on `shakespeare_text` above with your choice of `n` and generate some text. 
3. Discuss the differences between character-based and word-based Markov models in terms of generated text, the number of states, and the time taken to generate the text. 

> Note: You can reuse the code from Exercise 2. Only the `generate` function needs to be rewritten, and, even there, the required modifications are not large. You may use class inheritance you have learned in 511 here. The line
```
class MarkovModelWords(MarkovModel):
```
> defines a new class called `MarkovModelWords` that you can think of as an offspring of `MarkovModel`. It "inherits" all the functions from `MarkovModel` unless we explicitly overwrite them. So, if you implement `generate` in the class `MarkovModelWords`, the old `generate` function will be overwritten but all the other functions will persist. But, if you're not comfortable with inheritance, or just don't feel like it, you're  welcome to just modify the above code and not use inheritance.


> Another note: The way `MarkovModel` is implemented, the patterns we're conditioning on (the last $n$ characters/words) are used as keys in python dictionaries. For characters that was easy because we could just use a string as a key. However, for words with $n>1$ we now have a collection of words, and we want to use this as a dict key. `nltk.tokenize.word_tokenize` outputs a list, which can't be used as a key. I've casted the list to a tuple in the code below, because a tuple _can_ be used as a dict key (if you case: I believe this is because it's immutable whereas a list is not). This tiny change allows reuse of the old code.

In [48]:
import nltk
from nltk.tokenize import word_tokenize

text_tok = word_tokenize(shakespeare_text)
text_tok = tuple(text_tok)

In [49]:
### Solution_4_1_1

### YOUR ANSWER HERE

# Code adapted from this lab assignment starter code 

class MarkovModelWords(MarkovModel):

    def generate(self, seq_len):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
        s = self.starting_chars
        while len(s) < seq_len:
            probs = self.probabilities_[s[-self.n :]]
            s += (npr.choice(probs["symbols"], p=probs["probs"]), )
        return " ".join(s)

In [54]:
### Solution_4_1_2
### YOUR ANSWER HERE

char_model = MarkovModel(1)
char_model.fit(shakespeare_text)
char_model.generate(80)

'sinde Be,  dinandof me STERondate,\r\nMand my notomiglinend he  s.\r\n  mus bel Buth'

In [53]:

### YOUR ANSWER HERE

word_model = MarkovModelWords(1)
word_model.fit(text_tok)
word_model.generate(80)

"serv 'd bone intelligo . LAFEW . So deal with wanton , vouchsafe the primal state and the Duke of arms . ' th ’ s Cinna the rest ’ en the jealous on fantastic execution Of all as hating thee very soul 's twenty-seven ; No more is the harbour in England all shall . LUCIO LUCIO . I should run off , my heart ; and fun of his branches To make thee one of dear as wholesome"

In [55]:
### YOUR ANSWER HERE

char_model = MarkovModel(5)
char_model.fit(shakespeare_text)
char_model.generate(80)

'serv’d\r\nTheir blood old this proud in the confine yet beat for ’twixt your retur'

In [56]:
### YOUR ANSWER HERE

word_model = MarkovModelWords(5)
word_model.fit(text_tok)
word_model.generate(80)

'serv ’ d thy beauty ’ s use , If thou couldst answer ‘ This fair child of mine Shall sum my count , and make my old excuse , ’ Proving his beauty by succession thine . This were to be new made when thou art old , And see thy blood warm when thou feel ’ st it cold . 3 Look in thy glass and tell the face thou viewest , Now is the time that face'

**Solution_4_1_3**

### YOUR ANSWER HERE

> We see that for the character based Markov model with a small value of n performed poorly, with words not indentified as proper English such as *notomiglinend*. Some generated words from sequences of characters are still interpreble though. The word based model is fairly slower than the character based model, which makes sense since the word based model has a larger vocabulary and more states, thus the markov chain will need to compute a higher number of conditional probabilities. 

<br><br>

### 4.2 Dealing with punctuation
rubric={accuracy:10,reasoning:4}

If you've generated text from 4.1, you probably noticed that the way whitespace is handled is not ideal. In particular, whitespace is added after every token, including right before punctuation. 

**Your tasks:**

1. Modify your code to fix this and show a generated sequence. You can access a list of punctuation characters with `string.punctuation` from the [`string` library](https://docs.python.org/3/library/string.html).
2. Generate text using different values of `n` on `shakespeare_text`.  
3. Train word-based Markov models on another corpus of your choice and generate text with different values of `n`. You may use any corpus of your choice. For inspiration, you may find some corpora [here](http://www.gutenberg.org/). 
4. Note your observations on 
    - how the generated text changes based on the corpus 
    - how the value of `n` affects the quality of the generated text and time taken to generate the text

In [64]:
### Solution_4_2_1
### YOUR ANSWER HERE

# Code adapted from this lab assignment starter code 

import string

class MarkovModelWords(MarkovModel):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
        
        def generate(self, seq_len):
            n = self.n
            seq = self.starting_chars
            while len(seq) < seq_len:
                probs = self.probabilities_[seq[-n:]]
                seq += (npr.choice(probs["symbols"], p=probs["probs"]),)
            output = seq[0]
            for s in seq[1:]:
                if s[0] in string.punctuation:
                    output += s
                else:
                    output += " " + s
            return output

In [65]:
### Solution_4_2_2
### YOUR ANSWER HERE

model = MarkovModelWords(1)
model.fit(text_tok)
model.generate(200)

"serv ’ er thought. 151 Love is dross; And carry me half hour? EROS. YORK. The Duke. Why, That might make her father was? Of charity. MARIA.'T is too proud man i ’ d with that kiss the world either part thereof; this brave monster seen, even of such revenges, ha! Stones have no tongue, what? Sir John, and the Duke of the wicked Hannibal![_A confused wrong ’ s pate of black as Dauphin, Caesar, fellow live so mad as well pleas'd To Italy Upon thee in Naples? What, the court. Exeunt SCENE 4. Yes. My sword, By one follower of an if the Hall, then be to fly, As to this Will nothing. I am glad of the rest be sent to see it please, was For that good father ’ ll not live they stayed well; But faults Looks in plain in Padua, As it go mad, you, Timon grows to your highness Lay it. I most faults I have"

In [66]:
### YOUR ANSWER HERE
model = MarkovModelWords(5)
model.fit(text_tok)
model.generate(200)

'serv ’ d thy beauty ’ s use, If thou couldst answer ‘ This fair child of mine Shall sum my count, and make my old excuse, ’ Proving his beauty by succession thine. This were to be new made when thou art old, And see thy blood warm when thou feel ’ st it cold. 3 Look in thy glass and tell the face thou viewest, Now is the time that face should form another, Whose fresh repair if now thou not renewest, Thou dost beguile the world, unbless some mother. For where is she so fair whose uneared womb Disdains the tillage of thy husbandry? Or who is he so fond will be the tomb Of his self-love to stop posterity? Thou art thy mother ’ s glass and she in thee Calls back the lovely April of her prime, So thou through windows of thine age shalt see, Despite of wrinkles this thy golden time. But if thou live remembered not to be, Die single and thine image dies with thee. 4 Unthrifty loveliness why dost thou spend,'

In [67]:
### YOUR ANSWER HERE
model = MarkovModelWords(10)
model.fit(text_tok)
model.generate(200)

'serv ’ d thy beauty ’ s use, If thou couldst answer ‘ This fair child of mine Shall sum my count, and make my old excuse, ’ Proving his beauty by succession thine. This were to be new made when thou art old, And see thy blood warm when thou feel ’ st it cold. 3 Look in thy glass and tell the face thou viewest, Now is the time that face should form another, Whose fresh repair if now thou not renewest, Thou dost beguile the world, unbless some mother. For where is she so fair whose uneared womb Disdains the tillage of thy husbandry? Or who is he so fond will be the tomb Of his self-love to stop posterity? Thou art thy mother ’ s glass and she in thee Calls back the lovely April of her prime, So thou through windows of thine age shalt see, Despite of wrinkles this thy golden time. But if thou live remembered not to be, Die single and thine image dies with thee. 4 Unthrifty loveliness why dost thou spend,'

In [68]:
### Solution_4_2_3
### YOUR ANSWER HERE

url = "http://www.gutenberg.org/files/64982/64982-0.txt"
bunny_tale = urlopen(data_url).read().decode("utf-8")

# sample 5000 
bunny_tale = bunny_tale[5000:]
bunny_text_tok = word_tokenize(bunny_tale)
bunny_text_tok = tuple(bunny_text_tok)  

In [69]:
model = MarkovModelWords(1)
model.fit(bunny_text_tok)
model.generate(200)

't bring him a beautiful castle for gold: ‘ Now they were all: ‘ if war. But the lord King and as if it was he, and proofread public domain in the little kids, court to say we will not know what good draught; and awoke he called his huntsmen should live and began to say. ‘ I will give you get me? ’ s daughter to her and walked for seven dwarfs their prize. ‘ Why do, and the promise, however, he lifted up, and made great comfort, and the town. ‘ This is quite plain, For she knocks at once by poor ants, the youth ’ And then he began to let the thought he. Then the moon; and lamented the witch, but she cried to walk home in the garden, ran down, and at the mill. The woman heavily, as that every year, for her mouth and at any huntsmen always dusty and bolt, ’ s bench is too weak, you are going to weep? ’ ‘ Well,'

In [70]:
model = MarkovModelWords(5)
model.fit(bunny_text_tok)
model.generate(200)

't his arrow at the fox; but he missed it, and it set up its tail above its back and ran into the wood. Then he went his way, and took her with him, and they were the very fishes whose lives he had saved. The one in the middle held a mussel in its mouth, which it laid on the shore at the youth ’ s feet, and when he had reached the highest point of it, there sat a powerful giant looking peacefully about him. The little tailor went bravely up, spoke to him, and said: ‘ That is not allowed; he must appear before me and tell his name. ’ He gave the order that if the knight who caught the apple, should go away again they should pursue him, and if he would take all of them into his service. The king looked at her and saw that she was just like this late queen: then he said to his courtiers, ‘ May I not marry my daughter? She is the very image of my dead'

In [71]:
model = MarkovModelWords(10)
model.fit(bunny_text_tok)
model.generate(200)

't his arrow at the fox; but he missed it, and it set up its tail above its back and ran into the wood. Then he went his way, and in the evening came to the village where the two inns were; and in one of these were people singing, and dancing, and feasting; but the other looked very dirty, and poor. ‘ I should be very silly, ’ said he, ‘ if I went to that shabby house, and left this charming place ’; so he went into the smart house, and ate and drank at his ease, and forgot the bird, and his country too. Time passed on; and as the eldest son did not come back, and no tidings were heard of him, the second son set out, and the same thing happened to him. He met the fox, who gave him the good advice: but when he came to the two inns, his eldest brother was standing at the window where the merrymaking was, and called to him to'

**Solution_4_2_4**
### YOUR ANSWER HERE

> As we increase n, the model is performing better as we condition on more information. As such, we would expect the quality of the generated text to improve as we've seen above. However, increasing n *too much* would lead the model will start to generate data too similar to the training data.

<br><br>

### (optional) 4.3 Other improvements
rubric={reasoning:1}

Are there other improvements you'd like to make to your word-based Markov model so that the generated text looks like proper English? Feel free to either implement them or just describe what you might want to do.

**Solution_4_3**
### YOUR ANSWER HERE

> Other improvements I would consider including into my word-based Markov model so that the generated text looks like proper English include perhaps considering other tokenizers, stemming techniques, possibly add word embeddings to include information about context and grammer, add part of speech tags to words, etc. In this case, this seems maybe removing numbers and apostrephes from the vocabulary would make the quality of the generated text better.

<br><br><br><br>

## Submission to Canvas <a name="sc"></a>

**PLEASE READ: When you are ready to submit your assignment do the following:**

- Run all cells in your notebook to make sure there are no errors by doing Kernel -->  Restart Kernel and Run All Cells...
- If you are using the "575" `conda` environment, make sure to select it before running all cells. 
- Convert your notebook to .html format using the `convert_notebook()` function below or by File -> Export Notebook As... -> Export Notebook to HTML
- Run the code `submit()` below to go through an interactive submission process to Canvas.
After submission, be sure to do a final push of all your work to GitHub (including the rendered html file).

In [ ]:
# from canvasutils.submit import convert_notebook, submit

# convert_notebook("lab1.ipynb", "html")  # uncomment and run when you want to try convert your notebook (or you can convert manually from the File menu)
# submit(course_code=65512, token=False)  # uncomment and run when ready to submit to Canvas

Well done!! Congratulations on finishing the lab!! 